In [1]:
%reload_ext autoreload
%autoreload 2
import sys
sys.path.insert(1, '/p/project/chhb19/gaertner2/lkf_detection/functions/')
from statistics_functions import *
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
path = '/p/project/chhb19/gaertner2/data/awicm_cvmix/'

In [3]:
[years, lkfs, paths, paths_all] = np.load(path + 'data_lkfs_paths.npy', allow_pickle=True)

In [4]:
LKFs = np.load(path + 'LKFs.npy', allow_pickle=True)[0]

This is only the growth rate between two time steps of the tracked LKFs. It is not actually a good metric for the total LKF growth rate.

In [5]:
for year in years:
    
    yr = np.where(years==year)[0][0]
    for day in range(364):
        
        for i, lkf_path in enumerate(paths_all[yr][day]):
            
            if np.size(lkf_path)>1:
                LKFs[f'y{year}'][f'd{day}'].loc[i, 'growth rate'] =        \
                      LKFs[f'y{year}'][f'd{day+1}'].loc[lkf_path[1], 'length'] \
                    - LKFs[f'y{year}'][f'd{day}'].loc[i, 'length']
                
        if 'growth rate' not in LKFs[f'y{year}'][f'd{day}'].keys():
            LKFs[f'y{year}'][f'd{day}']['growth rate'] = np.nan
    
    LKFs[f'y{year}']['d364']['growth rate'] = np.nan

In [6]:
for year in years:
    total_growth_rate = []
    for day in range(365):
        total_growth_rate += np.nansum(LKFs[f'y{year}'][f'd{day}']['growth rate']),
    LKFs[f'y{year}']['total']['total growth rate'] = total_growth_rate

In [7]:
total_growth_rate = []
for year in years:
    total_growth_rate += LKFs[f'y{year}']['total']['total growth rate'],

In [8]:
tmp = av_sd(total_growth_rate,i=3)
LKFs['2010s']['total growth rate'] = tmp[0]
LKFs['2010s']['total growth rate sd'] = tmp[1]
LKFs['2090s']['total growth rate'] = tmp[2]
LKFs['2090s']['total growth rate sd'] = tmp[3]

In [9]:
res_km = 4.337849218906646

for year in years:
    yr = np.where(years==year)[0][0]

    for day in range(365):

        curv = []
        if (len(lkfs[yr][day])==0):
            LKFs[f'y{year}'][f'd{day}']['curvature'] = np.nan
        else:
            for i, lkf in enumerate(lkfs[yr][day]):
                start = np.array([lkf[0,0], lkf[0,1]], dtype='int')
                end = np.array([lkf[-1,0], lkf[-1,1]], dtype='int')

                distance = np.sqrt( (end-start)[0]**2 + (end-start)[1]**2 ) * res_km

                LKFs[f'y{year}'][f'd{day}'].loc[i,'curvature'] = 1 - distance / LKFs[f'y{year}'][f'd{day}'].loc[i, 'length']

In [10]:
for year in years:
    for day in range(365):
        LKFs[f'y{year}']['total'].loc[day,'mean curvature'] = np.nanmean(LKFs[f'y{year}'][f'd{day}']['curvature'])

In [11]:
i = 3
LKFs['2010s']['curvature'] = np.mean([LKFs[f'y{year}']['total']['mean curvature'] for year in years[:i]],axis=0)
LKFs['2010s']['curvature sd'] = np.sqrt(np.var([LKFs[f'y{year}']['total']['mean curvature'] for year in years[:i]],axis=0))
LKFs['2090s']['curvature'] = np.mean([LKFs[f'y{year}']['total']['mean curvature'] for year in years[i:]],axis=0)
LKFs['2090s']['curvature sd'] = np.sqrt(np.var([LKFs[f'y{year}']['total']['mean curvature'] for year in years[i:]],axis=0))

In [12]:
np.save(path + 'LKFs', [LKFs])